# ESPIDAM SIR Example

*Authors: Sebastiaan Weytjens, Pieter Libin, Niel Hens*

## SIR Models

### Theory

The diagram below shows the SIR model. The diagram shows the 3 compartments, S, I and R, and the rates between them. The rate between S and I is also refered to as the force of infection (FOI).

![The diagram of an SIR model.](sir_diagram.png)


To solve this model deterministically, we can formulate it as a set of Ordinary Differential Equations (ODEs),


$
\begin{aligned}
\frac{{dS}}{{dt}} &= -\beta \cdot I / N \cdot S \\
\frac{{dI}}{{dt}} &= \beta \cdot I / N \cdot S - \gamma \cdot I \\
\frac{{dR}}{{dt}} &= \gamma \cdot I
\end{aligned}
$

Alternatively, we can solve this model stochastically using a Binomial Chain [1], as shown below.

First, the new individuals in $I$ and $R$ are calculated, where $p^{S \rightarrow I}(t)$ and $p^{I \rightarrow R}(t)$ are probabilities of an individual transitioning from $S$ to $I$ or $I$ to $R$, respectively.

$
\begin{aligned}
I_{\text {new }} & \sim \operatorname{Binomial}\left(p^{S \rightarrow I}(t), S(t)\right) \\
R_{\text {new }} & \sim \operatorname{Binomial}\left(p^{I \rightarrow R}(t), I(t)\right) \\
\end{aligned}
$

The rate can be converted to a probability (e.g. $p^{S \rightarrow I}(t)$) as follows: 

$
\begin{aligned}
p &= 1-e^{-\text{rate}*dt}
\end{aligned}
$

Then, all compartments are updated accordingly.

$
\begin{aligned}
S(t+1) &= S(t) - I_{\text {new }} \\
I(t+1) &= I(t) + I_{\text {new }} - R_{\text {new }} \\
R(t+1) &= R(t) + R_{\text {new }}
\end{aligned}
$

[1]: Abrams, S., Wambua, J., Santermans, E., Willem, L., Kuylen, E., Coletti, P., Libin, P., Faes, C., Petrof, O., Herzog, S. A., Beutels, P., & Hens, N. (2021). Modelling the early phase of the Belgian COVID-19 epidemic using a stochastic compartmental model and studying its implied future trajectories. In Epidemics (Vol. 35, p. 100449). Elsevier BV. https://doi.org/10.1016/j.epidem.2021.100449

### Code

#### Dependencies

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import math
from sir_helpers import *

#### A function to convert a rate to a probability

In [ ]:
def rate_to_p(rate, dt):
    return 1 - math.exp(-rate * dt)

#### Model Description

In [ ]:
# Compartments
compartments = {
    "S": [],
    "I": [],
    "R": [],
}

#Transition rates
def foi(i, params, N):
    return i * params["beta"] / N

def i_r(params):
    return params["gamma"]

def initialise_modelstate(modelstate, seeds, N):
    modelstate["S"] = [N - seeds]
    modelstate["I"] = [seeds]
    modelstate["R"] = [0]

    return modelstate

#### Solvers for the model

Deterministically, using ODEs, or stochastically, using a Binomial Chain.

In [ ]:
# ODE Solver

def ode_system(y0, t, parameters):
        
    params = parameters["disease_params"]
    N = parameters["N"]

    s, i, r = y0
    ds, di, dr = 0, 0, 0

    #Calculating the difference
    diff = s * foi(i, params, N)
    ds -= diff
    di += diff

    diff = i * i_r(params)
    di -= diff
    dr += diff

    return ds, di, dr

def ode_solver(model_state, end_t, params, N):
    all_parameters = {
        "disease_params": params,
        "N": N
    }
    # Initial conditions (modelstates, timesteps)
    y0 = (model_state["S"][0], model_state["I"][0], model_state["R"][0])
    t = np.linspace(0, end_t, end_t)
    
    # Solving the ODE system
    ret = odeint(ode_system, y0, t, args=(all_parameters,))
    s, i, r = ret.T

    new_model_state = {
        "S": list(s),
        "I": list(i),
        "R": list(r)
    }

    return new_model_state

# Binomial Chain solver

def binom_solver(model_state, end_t, params, N, iterations):

    model_states = {
        "S": [],
        "I": [],
        "R": []
    }

    #For every iteration we simulate a trajectory
    for _ in range(iterations):

        s = [model_state["S"][0]]
        i = [model_state["I"][0]]
        r = [model_state["R"][0]]

        #we simulate 10 steps per day to get a more accurate approximation
        for t in range(1, end_t * 10):
            #Stochastically calculating the new individuals
            i_new = np.random.binomial(s[t-1], rate_to_p(foi(i[t-1], params, N), 1/10))
            r_new = np.random.binomial(i[t-1], rate_to_p(i_r(params), 1/10))

            s.append(s[t-1] - i_new)
            i.append(i[t-1] + i_new - r_new)
            r.append(r[t-1] + r_new)

        #Appending the iteration trajectories to the results
        indices = [index for index in range(0, end_t * 10, 10)]
        model_states["S"].append([s[index] for index in indices])
        model_states["I"].append([i[index] for index in indices])
        model_states["R"].append([r[index] for index in indices])

    return model_states

#### Model Parameters

In [ ]:
N = 1000000 #total population
seeds = 10 #initial number of infected individuals

R0 = 1.5 #basic reproduction number
gamma = 1/3 #recovery rate

params = {
    "beta": R0 * gamma, #transmission rate
    "gamma": gamma #recovery rate
}

end_t = 150 #days to simulate

#### Plotting

##### Plotting deterministic results

In [ ]:
modelstate = initialise_modelstate(compartments, seeds, N)
results = ode_solver(modelstate, end_t, params, N)

plot_ODE(results, "SIR Model (ODE Solver)")

##### Plotting stochastic results

Compare the results of the ODE with the means of the stochastic trajectories. Why are there differences?

In [ ]:
modelstate = initialise_modelstate(compartments, seeds, N)
results = binom_solver(modelstate, end_t, params, N, 100)

plot_binom(results, "SIR Model (Binomial Chain Solver)")

##### Comparing various $R_0$

$R_0$, also known as the basic reproduction number, represents the average number of secondary infections caused by a single infected individual in a completely susceptible population.

To calculate $R_0$ in this simple SIR model, we can use this formula:

$
R_0 = \beta / \gamma
$

In this example, we will compare trajectories for various $R_0$.

In [ ]:
results = {}

modelstate = initialise_modelstate(compartments, seeds, N)

for R0 in [0.75,1,1.25,1.5,2,2.5,3]:
    params = {
        "beta": R0*gamma, 
        "gamma": gamma
    }

    results[R0] = binom_solver(modelstate, end_t, params, N, 100)

plot_binom_R0s(results, "SIR Model (Binomial Chain) - Varying R0 - With Errors")